In [1]:
%matplotlib inline

# MLP

Uma das formas mais tradicionais de redes neurais é a MLP, ou Multilayer Perceptron. 
A MLP é um algoritmo de aprendizado supervisionado que aprende uma função que relaciona entradas e saídas através do seu treinamento em um conjunto de dados.

Tipicamente, uma MLP é formada por:
- uma camada de entrada, que corresponde aos M atributos a serem usados para a classificação
- uma camada de saída, que possui um neurônio para cada uma das N classes em que a amostra de atributos será classificada
- uma ou mais camadas ocultas de neurônios, responsáveis por tornar a MLP um classificador não linear



## A instanciação de uma MLP

Para criarmos uma MLP no `scikit-learn` usamos a classe `MLPClassifier` do pacote `sklearn.neural_network` através do código:
```python
from sklearn.neural_network import MLPClassifier
```
A instanciação de um objeto representando a MLP depende da definição de **hiperparâmetros**, ou seja parâmetros da rede que não serão aprendidos durante a fase de treinamento. Até agora, os hiperparâmetros que aprendemos são:

### 1. Número de camadas ocultas
Indica quandas camadas ocultas serão usadas na rede neural. Há provas matemáticas de que uma rede do tipo MLP com função de ativação sigmóide pode aproximar qualquer função contínua com apenas uma única camada oculta de neurônios, porém para funções descontínuas, duas são necessárias (ver livro do Russel, pág. 720). Na prática, para dados simples, não precisamos de mais do que uma camada, e partimos para a decisão do número de neurônios nessa camada. O inconveniente dessa abordagem é que o número de neurônios necessários para realizar a classificação corretamente pode ter que crescer muito dependendo da complexidade dos dados. 

### 2. Número de neurônios por camada oculta
No caso de escolhermos usar apenas uma camada oculta, é preciso escolher o número de neurônios a ser empregado.
Há algumas "receitas de bolo" para essa decisão, porém a melhor forma de determinar esse parâmetro é realizar diversos testes com os dados de treinamento, através da *validação cruzada*. Uma dessas receitas de bolo indica que o número de neurônios da camada oculta seja o dobro do número de elementos de entrada mais um, que nós podemos usar como ponto de partida para a realização de testes.

### 3. Função de ativação $f(a)$
A função de ativação processa a entrada líquida do neurônio, resultante do somatório das entradas multiplicadas pelos pesos.
Para a tarefa de classificação, as funções de ativação mais comumente empregadas são:
- Logística ou sigmóide: $f(a)=\frac{1}{1+e^{-a}}$
<img width="400px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1920px-Logistic-curve.svg.png"/>
- Tangente hiperbólica: $f(a) = tanh(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}}$
<img width="400px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Hyperbolic_Tangent.svg/250px-Hyperbolic_Tangent.svg.png">

- Linear retificada (ReLU): $f(a) = \begin{cases} a, & \mbox{se } a>=0 \\ 0, & \mbox{caso contrário } \end{cases}$
<img width="400px" src="http://ml4a.github.io/images/figures/relu.png">


Há também parâmetros relacionados ao processo de treinamento:
- Taxa de aprendizado $\eta$: valor maior do que zero que representa a velocidade em que ocorre o aprendizado
- *Batch size*: número de amostras a serem usadas a cada atualização de pesos
- Algoritmo de otimização: embora o algoritmo Backpropagation e suas variações seja o mais empregado, há outros algoritmos disponíveis para o trienamento. Aqui vamos trabalhar com o Backpropagation tradicional.
- Número de épocas: determina quantas épocas, ou seja, quantas vezes cada amostra de entrada será utilizada, mesmo que o algoritmo de otimização não tenha encontrado um resultado confiável

Dessa forma, a instanciação da nossa rede MLP fica:

In [2]:
from sklearn.neural_network import MLPClassifier
# Definição dos hiperparâmetros
# Função de ativação: pode ser uma dentre: {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’
activation = 'logistic' 
# Tamanhos das camadas ocultas
hidden_layer_sizes = [100] # Apenas uma camada com cem neurônios
# Algoritmo de otimização: pode ser um dentre {‘lbfgs’, ‘sgd’, ‘adam’}, default ‘adam’
solver = 'sgd' #Stochastic Gradient Descent
# Valor da taxa de aprendizado, default 0.001
learning_rate =  0.03
# Batch size, default é 'auto', ou seja, min(200, n_samples)
batch_size=20
# Número máximo de épocas, default 200
max_iter = 1000
mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                    activation=activation, solver=solver,
                    learning_rate_init=learning_rate,
                    batch_size=batch_size,
                    max_iter=max_iter)

# Classificando Imagens com a MLP

Na classificação de padrões de imagens, nosso objetivo é classificar o conteúdo de imagens digitais. Por exemplo, podemos segmentar os caracteres na imagem de um documento, e, para cada imagem de caractere, identificar qual é a letra ou dígito correspondente. Outro exemplo é identificar qual animal está presente na imagem (*cachorro*, *gato* ou *papagaio*, por exemplo).


## 1. Gerando o vetor de atributos

Para classificar o conteúdo de imagens digitais, uma abordagem muito comum é o uso dos seus próprios pixels como atributos. Assim, se uma imagem tem tamanho 100x100 pixels, terermos um vetor de atributos de 10.000 posições!
- Vantagem: ganhamos flexibilidade no processo de classificação, já que não precisamos nos preocupar com a especificação de atributos específicos
- Desvantagem: grande número de atributos, torna o treinamento muito lento e pode inclusive não finalizar em tempo hábil 

Assim vamos definir a função `gera_vetor(w, h, arq)` que toma um arquivo de imagem, faz um preprocessamento e gera o vetor de atributos. Uma vez que o vetor de atributos deve semrpe ter o mesmo tamanho, a imagem é reescalonada para `h` linhas (o que corresponde à altura, ou * **h**eight*) e `w` colunas (o que corresponde à sua largura, ou * **w**idth*) 


In [3]:
import cv2
import numpy as np

def gera_vetor(w, h, arq):
    """
    w: largura desejada para a imagem
    h: altura desejada para a imagem
    arq: Caminho do arquivo de imagem
    retorno: array com o vetor de atributos, de tamanho w*h
    """
    # Ler imagem como tons de cinza
    img = cv2.imread(arq, cv2.IMREAD_GRAYSCALE)
    # Pré-processamento: 
    # 1. Passa o filtro da mediana
    # 2. gera imagem da magnitude do gradiente
    # 3. Faz a média ponderada da imagem com o resutado da magnitude do gradiente
    img = cv2.medianBlur(img, 3)
    # O tipo de imagem CV_16 comporta pixels de 16bits com valores positivos e negativos
    sobelx = cv2.Sobel(img, cv2.CV_16S, 1, 0)
    sobely = cv2.Sobel(img, cv2.CV_16S, 0, 1)
    # Retorna ao tipo original de 8 bits unsigned (0 a 255)
    mag_grad = cv2.add(np.abs(sobelx), np.abs(sobely), dtype=cv2.CV_8U)
    img = cv2.addWeighted(img, 0.7, mag_grad, 0.3, gamma=0)
    
    #Redimensionando a imagem
    img = cv2.resize(img, (w, h))
    
    # Transforma a imagem img em um array unidimensional e retorna
    return img.ravel()
        

## 2. Criando a matriz de dados de treinamento

Para usar os algoritmos de classificação de padrões é necessário transformar as amostras de vetor de atributos em linhas da matriz de treinamento.

Vamos assumir que as imagens representativas das diferentes classes estão armazenadas em diretórios homônimos. Assim, para cada imagem em cada diretório, iremos criar uma linha da matriz e uma entrada no array de classes, que chamaremos de `alvos`.

In [4]:
import os
import numpy as np
classes = ["cachorro", "gato", "papagaio"]

# São os valores de altura e largura originais das imagens MNIST
altura = 28
largura = 28
treinamento = []
alvos = []
for cls in classes:
    for file in os.listdir(cls):
        treinamento.append( gera_vetor(largura, altura, "{}/{}".format(cls,file)) )
        alvos.append(cls)

#Transforma em array
treinamento = np.array(treinamento)
alvos = np.array(alvos)

print(treinamento.shape, alvos.shape)

(70, 784) (70,)


## 3. Reduzindo a dimensionalidade dos dados


Um grande número de atributos pode atrapalhar o treinamento de classificadores, em especial a MLP.
Para melhorar o desempenho do treinamento, vamos trabalhar os dados através da sua normalização e da redução da dimensionalidade.

Reduzir a dimensionalidade significa diminuir as dimensões do vetor de atributos tentando manter a qualidade da informação ali contida. Uma das formas de fazer isso é através do PCA - *Principal Component Analysis* ou Análise de Componentes Principais.

In [5]:
# Normalizando os dados de treinamento
media = treinamento.mean(axis=0)
desvio = treinamento.std(axis=0)
treinamento_norm = (treinamento - media)/desvio

#Reduzindo a dimensionalidade
from sklearn.decomposition import PCA
# Instancia o modelo PCA de forma que 85% da variabilidade de dados seja mantida
pca = PCA(.85)
# Calcula o modelo e transforma os dados de treinamento
treinamento_pca = pca.fit_transform(treinamento_norm)
# Verificando o novo número de atributos
print("Novo número de atributos: ", treinamento_pca.shape[1])

Novo número de atributos:  26


## 4. Avaliando o resultado da classificação 

Para podermos validar o resultado da classificação, temos que separar algumas amostras de treinamento para que sejam usadas para teste, sem utilizá-las no treinamento da rede neural. Assim, podemos comparar o resultado desse teste com as classes reais esperadas.

In [6]:
# 3. Separando as amostras de treinamento e validação
frac_validacao = 0.2 # Separamos 20% para a validação
# Primeiro embaralhamos os indices da matriz
import numpy.random as random
indices = random.permutation(treinamento_pca.shape[0])

# ... depois separamos em porção de treinamento e validação
import math
icut = math.floor(frac_validacao * indices.shape[0])
indices_treino = indices[icut:]
indices_valid = indices[:icut]


## 5. Treinando e executando a rede neural criada

In [7]:
# 4. Treinar o classificador MLP instanciado anteriormente, seprando as entradas e saídas 
mlp.fit(treinamento_pca[indices_treino,:], alvos[indices_treino])

MLPClassifier(activation='logistic', alpha=0.0001, batch_size=20, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[100], learning_rate='constant',
       learning_rate_init=0.03, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Uma observação sobre a sintaxe dos classificadores do `scikit-learn`
- O método fit(X,Y) recebe uma matriz ou dataframe X onde cada linha é uma amostra de aprendizado, e um array Y contendo as saídas esperadas do classificador, seja na forma de texto ou de inteiros
- O método predict(X) recebe uma matriz ou dataframe X onde cada linha é uma amostra de teste, retornando um array de classes

In [8]:
# 5. Executar o classificador nas amostras de validação
classes_valid = mlp.predict(treinamento_pca[indices_valid,:])

#Cálculo do erro
erro = np.sum(classes_valid != alvos[indices_valid])/indices_valid.shape[0]
print("Erro médio de classificação: ", erro)

Erro médio de classificação:  0.7142857142857143
